# Aula 3: Embeddings de Alta Performance

## O que vamos aprender:
- Comparar modelos de API (Gemini) vs. modelos locais (Hugging Face) em custo, velocidade e qualidade.
- Aumentar a eficiência do processamento em 10x com **Batch Processing**.
- Economizar custos e reduzir a latência com **Cache de Embeddings**.

### Por que a performance dos embeddings é crucial?
- **Qualidade da Busca**: A precisão do seu RAG depende diretamente da qualidade dos embeddings.
- **Custo Operacional**: Modelos locais podem reduzir drasticamente os custos de API em larga escala.
- **Velocidade (Latência)**: O tempo para gerar embeddings impacta a velocidade de indexação e a resposta ao usuário.
- **Privacidade**: Modelos locais garantem que dados sensíveis permaneçam na sua infraestrutura.

## 0. Configuração

Instalamos as bibliotecas e configuramos a chave de API do Google.

In [1]:
!pip install langchain==0.3.26 langchain-community==0.3.27 langchain-core==0.3.71 langchain-google-genai==2.1.8 sentence-transformers==4.1.0 scikit-learn==1.6.1

In [2]:
import os
import time
import numpy as np

os.environ['GOOGLE_API_KEY'] = '...'

## 1. Comparativo de Modelos: Gemini (API) vs. Hugging Face (Local)

Vamos comparar um modelo de ponta via API (Google Gemini) com modelos open-source populares que rodam localmente.

- **Google Gemini (`embedding-001`)**: Modelo de alta qualidade, acessado via API.
- **`all-MiniLM-L6-v2`**: Um modelo local muito popular, leve e rápido. Ótimo para tarefas gerais onde a velocidade é importante.
- **`BAAI/bge-large-en-v1.5`**: Um dos melhores modelos open-source no MTEB Leaderboard. Mais pesado, mas com qualidade semântica superior.

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import time

In [4]:
# Textos de exemplo para nosso teste
textos_teste = [
    "Qual é a política de férias da nossa empresa?",
    "Preciso de um relatório de despesas de viagem.",
    "Como configuro o acesso à rede privada virtual (VPN)?",
    "Onde encontro o código de conduta da organização?",
    "Quero entender o processo de avaliação de performance."
]

In [5]:
## Google Gemnin Embeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

start_time = time.time()
embeddings_gemini = gemini_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time} segundos")
print(f"  - Dimensões do vetor: {len(embeddings_gemini[0])}")

Tempo de processamento: 0.3485264778137207 segundos
  - Dimensões do vetor: 3072


In [6]:
#embeddings_gemini

[GoogleGenerativeAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/)

In [7]:
## all-MiniLM-L6-v2

minilm_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
start_time = time.time()
embeddings_minilm = minilm_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time} segundos")
print(f"  - Dimensões do vetor: {len(embeddings_minilm[0])}")

/tmp/ipython-input-1442496789.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  minilm_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Tempo de processamento: 0.2903017997741699 segundos
  - Dimensões do vetor: 384


In [8]:
print(f"  - Dimensões do vetor: {len(embeddings_minilm[0])}")

  - Dimensões do vetor: 384


[HuggingFaceEmbeddings](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html)

[all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
)

In [9]:
#embeddings_minilm

[[-0.036897484213113785,
  0.04317065700888634,
  -0.10045938938856125,
  -0.06489814817905426,
  -0.07398222386837006,
  0.025539880618453026,
  0.012299592606723309,
  -0.006687232758849859,
  0.038479339331388474,
  0.009164758957922459,
  0.040292561054229736,
  -0.025638654828071594,
  0.0021347426809370518,
  -0.03667202219367027,
  -0.02804563008248806,
  -0.04484323412179947,
  -0.04096517339348793,
  0.03963225334882736,
  0.0352117121219635,
  0.06695879250764847,
  0.02319934032857418,
  -0.11209537088871002,
  -0.07564574480056763,
  0.00898084044456482,
  -0.020141301676630974,
  -0.000983282458037138,
  -0.012692359276115894,
  0.011275770142674446,
  -0.041977714747190475,
  -0.0676041692495346,
  0.04990863800048828,
  -0.03165162354707718,
  0.04525789991021156,
  -0.01792217791080475,
  0.009165960364043713,
  -0.006927618756890297,
  0.07252564281225204,
  -0.034365132451057434,
  0.030729813501238823,
  0.023424938321113586,
  -0.0639537125825882,
  0.00645322026684

In [10]:
# BGE-Large

bge_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

start_time = time.time()
embeddings_bge = bge_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time} segundos")
print(f"  - Dimensões do vetor: {len(embeddings_bge[0])}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Tempo de processamento: 2.8938069343566895 segundos
  - Dimensões do vetor: 1024


[bge-large-en-v1.5](https://huggingface.co/BAAI/bge-large-en-v1.5)

### Análise de Qualidade Semântica

Agora, vamos ver qual modelo entende melhor uma pergunta semanticamente similar, mas com palavras diferentes.

In [11]:
pergunta = "Quero tirar uns dias de folga do trabalho."

emb_pergunta_gemini = gemini_embeddings.embed_query(pergunta)
emb_pergunta_minilm = minilm_embeddings.embed_query(pergunta)
emb_pergunta_bge = bge_embeddings.embed_query(pergunta)

In [12]:
modelos = {
    "Gemini": (emb_pergunta_gemini, embeddings_gemini),
    "MiniLM": (emb_pergunta_minilm, embeddings_minilm),
    "BGE-large": (emb_pergunta_bge, embeddings_bge)
}

print(pergunta)

Quero tirar uns dias de folga do trabalho.


In [13]:
for nome, (emb_q, emb_docs) in modelos.items():
  similaridades = cosine_similarity([emb_q], emb_docs)[0]
  doc_e_similaridade = sorted(
      zip(textos_teste, similaridades), key=lambda x: x[1], reverse=True
  )
  print(f"--- Ranking para o modelo {nome} ---")
  for i, (doc, sim) in enumerate(doc_e_similaridade[:3], 1):
    print(f"  {i}. (Score: {sim:.3f}) {doc}")
  print()

--- Ranking para o modelo Gemini ---
  1. (Score: 0.831) Qual é a política de férias da nossa empresa?
  2. (Score: 0.785) Preciso de um relatório de despesas de viagem.
  3. (Score: 0.746) Quero entender o processo de avaliação de performance.

--- Ranking para o modelo MiniLM ---
  1. (Score: 0.496) Quero entender o processo de avaliação de performance.
  2. (Score: 0.469) Onde encontro o código de conduta da organização?
  3. (Score: 0.465) Qual é a política de férias da nossa empresa?

--- Ranking para o modelo BGE-large ---
  1. (Score: 0.646) Qual é a política de férias da nossa empresa?
  2. (Score: 0.645) Preciso de um relatório de despesas de viagem.
  3. (Score: 0.621) Onde encontro o código de conduta da organização?



 **o BGE-large foi o único modelo que colocou a frase sobre férias (“Qual é a política de férias…”) em 1.º lugar**, exatamente o que esperamos quando a consulta é “Quero tirar uns dias de folga do trabalho.”

| Modelo        | Top-1 retornado                                            | Por que não/por que sim?                                                                 |
| ------------- | ---------------------------------------------------------- | ---------------------------------------------------------------------------------------- |
| **Gemini**    | *“Quero entender o processo de avaliação de performance.”* | O modelo pareceu priorizar a parte “Quero …” e ignorou o tema “folga/férias”.            |
| **MiniLM**    | *“Quero entender o processo de avaliação de performance.”* | Patinou de forma parecida com o Gemini; scores baixos (\~0.49) indicam incerteza.        |
| **BGE-large** | *“Qual é a política de férias da nossa empresa?”*          | Captou corretamente o conceito “folga/férias” e trouxe o item mais relevante como Top-1. |

### Por que isso acontece?

* **Treinamento/Foco linguístico**

  * *BGE-large* (BAAI) foi ajustado para “embedding for retrieval” e costuma ser forte em captar semântica, mesmo fora do inglês.
  * *Gemini embeddings* ainda estão em pré-lançamento e podem favorecer estruturas de frase (“Quero…”) mais do que o tema.
  * *MiniLM* é leve e rápido, mas perde profundidade semântica em queries curtas e fora do domínio do treinamento.

* **Tamanho e capacidade**
  Modelos maiores (BGE-large ≈ 1 B parâmetros) têm mais “espaço” para nuances semânticas em comparação ao MiniLM-L6-v2 (\~33 M).

* **Idioma misto (PT) e dados de treino**
  Nem todos os modelos foram igualmente expostos a português. BGE-large costuma lidar melhor com multilíngue que MiniLM; Gemini ainda pode variar.

### Cuidado ao tirar conclusões

* **Uma única query não faz estatística** – para bater o martelo, rode um conjunto maior de perguntas e compute métricas como *Top-k accuracy* ou *Mean Reciprocal Rank*.
* **Scores não são comparáveis entre modelos** – 0.82 do Gemini ≠ 0.64 do BGE; vale apenas a *ordem* dentro do próprio modelo.
* **Combine com filtros/feedback** – se a qualidade for crítica, você pode usar rerankers (e.g. Cohere Rerank, bge-reranker) sobre o Top-k retornado.

### O que fazer na prática?

1. **Teste com várias consultas reais** do seu domínio.
2. **Avalie recall\@k** (quantas vezes o documento correto aparece no Top-k).
3. Se BGE-large mantiver o desempenho, ele é um ótimo candidato como modelo de embeddings – ainda mais rodando localmente, sem custo de API.

Assim, para esta consulta específica, o BGE-large realmente se saiu melhor, mas faça um benchmark mais amplo antes de decidir pelo modelo definitivo.




---



---



## 2. Caching de Embeddings: Economia e Velocidade

Gerar embeddings, especialmente via API, tem custos de tempo e dinheiro. O cache armazena os embeddings já calculados para evitar retrabalho. Usaremos o `CacheBackedEmbeddings` do LangChain.

In [14]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

store = LocalFileStore("./cache/")

embedder_principal = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder_principal,
    store,
    namespace='gemini_cache'
)

/usr/local/lib/python3.12/dist-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA‑1 is *not* collision‑resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA‑256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


In [19]:
textos_para_cache = ["Olá, mundo!", "Testando o cache de embeddings.", "Olá, mundo!"]

start_time = time.time()
embeddings_result_1 = cache_embeddings.embed_documents(textos_para_cache)
end_time = time.time()

print(f"  - Tempo de execução: {end_time - start_time:.4f} segundos.")

  - Tempo de execução: 0.0146 segundos.


## 3. Batch Processing para Indexação em Larga Escala

Ao indexar milhares de documentos, processá-los em lotes (batches) é fundamental. Modelos locais, especialmente, se beneficiam enormemente disso.

In [20]:
documentos_grandes = [f"Este é o documento de teste número {i}." for i in range(1000)]

bge_embedder = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

batch_sizes = [1, 32, 64, 128]

In [22]:
for batch_size in batch_sizes:
    start_time = time.time()
    num_batches = len(documentos_grandes) // batch_size
    tempo_estimado = num_batches * (0.1 * batch_size) + (len(documentos_grandes) % batch_size) * 0.1
    tempo_real = bge_embedder.client.encode(documentos_grandes, batch_size=batch_size)
    end_time = time.time()

    print(f"  - Batch Size: {batch_size:<4} -> Tempo: {end_time - start_time:.2f}s")

  - Batch Size: 1    -> Tempo: 431.49s
  - Batch Size: 32   -> Tempo: 187.42s
  - Batch Size: 64   -> Tempo: 178.79s
  - Batch Size: 128  -> Tempo: 175.05s


## 📚 Resumo Prático da Aula 3

- **Qualidade vs. Custo**: Modelos de API como o Gemini oferecem alta qualidade com zero setup, mas a um custo por chamada. Modelos locais como o `BGE-large` oferecem qualidade comparável com custo de infraestrutura, além de privacidade.
- **Eficiência é Chave**: `CacheBackedEmbeddings` é uma ferramenta poderosa para economizar dinheiro e tempo, eliminando chamadas redundantes.
- **Escalabilidade**: Para indexar grandes volumes, o processamento em lote (batch processing) não é opcional, é uma necessidade.
